# 프로세스 설명
1. RAG를 LLM으로 개체와 관계를 추출 ->  지식 그래프를 구축

2. 질문의 개체와 엔티티에서 정확히 일치하는 개체를 그래프에서 찾음.

3. 정확히 일치하는 개체가 있다면 해당 개체로 검색. 없다면 벡터 유사도가 임계치 이상인 엔티티들로 검색

4. 정확히 일치하는 관계가 있다면 해당 관계로 검색. 없다면 벡터 유사도가 임계치 이상인 관계로 검색

5. 검색된 개체 또는 관계의 메타데이터에 포함된 페이지 번호로 실제 문서에서 검색된 개체 또는 관계가 포함된 문장을 찾고, 앞뒤에 있는 n문장을 LLM에 입력

In [ ]:
# import os
# import torch
# from langchain.schema import Generation, LLMResult
# from langchain.llms.base import LLM
# from model_loader.config import generation_loader
# from langchain_core.documents import Document
# from model_loader.local_loader import LocalModelLoader
# from langchain_experimental.graph_transformers import LLMGraphTransformerj

# def extract_entities_and_relations(input_dir, output_dir, pages_to_process=5):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     graph_generator = LLMGraphTransformer(llm=CustomLLM(generation_loader=generation_loader), strict_mode=False)

#     for filename in os.listdir(input_dir):
#         if filename.endswith(".md"):
#             filepath = os.path.join(input_dir, filename)
#             print(f"Processing {filepath}...")
#             with open(filepath, "r", encoding="utf-8") as f:
#                 content = f.read()

#             pages = content.split('\n\n') 
            
#             extracted_text = []
#             for i, page_content in enumerate(pages[:pages_to_process]):
#                 if not page_content.strip():
#                     continue

#                 try:
#                     documents = [Document(page_content=page_content)]
                    
#                     graph_documents = graph_generator.convert_to_graph_documents(documents)

#                     for graph_document in graph_documents:
#                         extracted_text.append(f"--- Page {i+1} ---")
#                         extracted_text.append("Entities:")
#                         for entity in graph_document.nodes:
#                             extracted_text.append(f"  - {entity.type}: {entity.id}")
                        
#                         extracted_text.append("\nRelationships:")
#                         for relationship in graph_document.relationships:
#                             extracted_text.append(f"  - ({relationship.source.id})-[{relationship.type}]->({relationship.target.id})")
#                         extracted_text.append("\n")

#                 except Exception as e:
#                     print(f"Error processing page {i+1} of {filename}: {e}")
#                     extracted_text.append(f"--- Error processing Page {i+1}: {e} ---")
                
#             output_filename = os.path.splitext(filename)[0] + "_extracted.txt"
#             output_filepath = os.path.join(output_dir, output_filename)
#             with open(output_filepath, "w", encoding="utf-8") as out_f:
#                 out_f.write("\n".join(extracted_text))
#             print(f"Extraction for {filename} completed. Results saved to {output_filepath}")

# class CustomLLM(LLM):
#     generation_loader: object
#     is_local_model: bool = False
#     llm_model: object = None
#     llm_tokenizer: object = None

#     def __init__(self, generation_loader, **kwargs):
#         super().__init__(generation_loader=generation_loader, **kwargs) 
#         self.generation_loader = generation_loader
#         self.is_local_model = isinstance(self.generation_loader, LocalModelLoader)
#         if self.is_local_model:
#             self.llm_model = self.generation_loader.model
#             self.llm_tokenizer = self.generation_loader.tokenizer

#     def _call(self, prompt: str, stop=None) -> str:
#         if self.is_local_model:
#             inputs = self.llm_tokenizer(prompt, return_tensors="pt").to(self.llm_model.device)
#             max_new_tokens = 512

#             with torch.no_grad():
#                 outputs = self.llm_model.generate(
#                     **inputs,
#                     max_new_tokens = max_new_tokens,
#                     num_return_sequences = 1,
#                     do_sample = True,
#                     temperature = 0.4,
#                     top_p = 0.9,
#                     repetition_penalty = 1.2,
#                     eos_token_id = self.llm_tokenizer.eos_token_id
#                 )
#             generated_text = self.llm_tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
#         else:
#             generated_text = self.generation_loader.generate(prompt)
#         return generated_text

#     @property
#     def _llm_type(self) -> str:
#         return "custom_generation_loader_llm"

# if __name__ == "__main__":
#     input_directory = "./data/split_file/anatomy"
#     output_directory = "./data/extracted_results/LLMGraphTransformer"
#     extract_entities_and_relations(input_directory, output_directory, pages_to_process=5)


Processing ./data/split_file/anatomy/1_Embryology.md...
Extraction for 1_Embryology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/1_Embryology_extracted.txt
Processing ./data/split_file/anatomy/2_Osteology.md...
Extraction for 2_Osteology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/2_Osteology_extracted.txt
Processing ./data/split_file/anatomy/3_Syndesmology.md...
Extraction for 3_Syndesmology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/3_Syndesmology_extracted.txt
Processing ./data/split_file/anatomy/4_Myology.md...
Extraction for 4_Myology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/4_Myology_extracted.txt
Processing ./data/split_file/anatomy/5_Angiology.md...
Extraction for 5_Angiology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/5_Angiology_extracted.txt
Processing ./data/split_file/anatomy/9_Neurology.md...
Extraction for 9_Ne

: 

# 개체-관계 추출

In [ ]:
import os
import torch
import re 
from langchain.schema import Generation, LLMResult
from langchain.llms.base import LLM
from model_loader.config import generation_loader
from langchain_core.documents import Document
from model_loader.local_loader import LocalModelLoader
from langchain_experimental.graph_transformers import LLMGraphTransformer

# MY_NODE_TYPES = ['ADJECTIVE', 'ALIAS', 'ANATOMICAL_FORAMEN', 'ANATOMICAL_GROUP', 'ANATOMICAL_LANDMARK', 'ANATOMICAL_PROCESS', 'ANATOMICALREGION', 'ANATOMICALSPACE', 'ANATOMICAL_SURFACE', 'ANATOMICAL_TERM', 'ANATOMICALSTRUCTURE', 'ANATOMY', 'ANIMAL', 'APONEUROSIS', 'ARTIFACT', 'AXIS', 'BIOLOGICAL_MATERIAL', 'BIOLOGICAL_PROCESS', 'BIOLOGICALENTITY', 'BIOLOGICALPROCESS', 'BODYFLUID', 'BONE', 'BONE_AXIS', 'BONE_DESCRIPTION', 'BONE_SYSTEM', 'BONE_TISSUE', 'BONE_TYPE', 'BORDER', 'BOUNDARY', 'BRAIN_STRUCTURE', 'BRAINSTEM', 'CANAL', 'CAPSULE', 'CARTILAGE', 'CELL', 'CELL_TYPE', 'CELLSTRUCTURE', 'CELLTYPE', 'CHARACTERISTIC', 'CHEMICAL', 'CIRCLE', 'CIRCULATION', 'CIRCULATORY_SYSTEM', 'CREST', 'CURVE', 'DATA', 'DATE', 'DEPRESSION', 'DESCRIPTION', 'DIRECTION', 'DISK', 'DIVERTICULUM', 'DUCT', 'DYE', 'ELEMENT', 'EMINENCE', 'ENVIRONMENT', 'EVENT', 'EXTREMITY', 'FASCIA', 'FIBER', 'FIELD_OF_STUDY', 'FLUID', 'FORAMEN', 'FORMATION', 'GENDER', 'GEOMETRICPROPERTY', 'GLAND', 'GROOVE', 'GROUP', 'INFORMATION', 'JOINT', 'JUNCTION', 'LAW', 'LAYER', 'LIFE_STAGE', 'LIGAMENT', 'LINE', 'MANNER', 'MATERIALPROPERTY', 'MEATUS', 'MECHANISM', 'MEDICALCONDITION', 'MEMBRANE', 'MOLECULE', 'MUSCLE', 'MUSCLE_ACTION', 'NERVE', 'NERVOUS_SYSTEM', 'NETWORK', 'NODE', 'NOMENCLATURE', 'NOTCH', 'OBSERVATION', 'ORGAN', 'ORGAN_SYSTEM', 'ORGANELLE', 'ORGANISM', 'ORIGIN', 'PATH', 'PERSON', 'PHYSICAL_CONTACT', 'PHYSICALACTION', 'PHYSICALPROPERTY', 'PHYSICALSUPPORT', 'PLANE', 'PLATE', 'POINT', 'PROCESS', 'PROFESSION', 'PROPERTY', 'PROTEIN', 'PUBLICATION', 'QUANTITY', 'RIDGE', 'SECTION', 'SEPTUM', 'SHAPE', 'SINUS', 'SKIN', 'SPATIALARRANGEMENT', 'SPINE', 'STATE', 'SUBSTANCE', 'SURFACE', 'SYSTEM', 'TENDENCY', 'TENDON', 'THEORY', 'TISSUE', 'TOOL', 'TOOTH', 'TOPIC', 'TUBEROSITY', 'VALVE', 'VESICLE', 'VESSEL', 'WORK', 'ANATOMY', 'BODY_PART', 'LIGAMENT', 'NERVE']

# MY_RELATIONSHIP_TYPES = ['ABDUCT', 'ABSORB', 'ACCOMPLISH', 'ACCUMULATE', 'ACTION', 'ADAPT', 'ADMIT', 'AFFECT', 'ALLOW', 'ANTAGONIZE', 'APPEAR', 'ARCHES_OVER', 'ARCH_FORWARD', 'ARE', 'ATTACH', 'BRING', 'CHARACTERIZE', 'ESTABLISH', 'EXPAND', 'FORM', 'FUSE', 'MAKE', 'OCCUPY', 'ORIFICE', 'OSSIFY', 'REPLACE', 'ARISE', 'ARRANGE', 'ARTICULATE', 'ASSIST', 'ASSUME', 'ATTACHMENT', 'AUTHORED', 'AUTHOR_OF', 'SACROCOCCYGEAL_LIGAMENT_POSTERIOR', 'BEAR', 'BEGIN', 'BELIEVE', 'BELOW', 'BEND', 'BIND', 'BLEND', 'BOUND', 'BREAK', 'BRIDGE', 'DO', 'CAPABLE_OF', 'CARRY', 'CAUSE', 'CHANGE', 'CHECK', 'CLOSE', 'CLOTH', 'COME', 'COMMENCE', 'COMMUNICATE', 'COMPLETE', 'COMPOSE', 'COMPRESS', 'CONNECT', 'CONSTITUTE', 'CONTINUE', 'CONTRACT', 'CONTRIBUTE', 'CONVERGE', 'CONVERT', 'CORRESPOND', 'COVER', 'CREATE', 'CROSS', 'CURVE', 'CUT', 'SACROCOCCYGEAL_LIGAMENT_LATERAL', 'PUBIC_ARCH_FORMATION', 'SUPERIOR_PUBIC_LIGAMENT', 'DECREASE', 'DEEPEN', 'DEGENERATE', 'DEPOSIT', 'DERIVE_FROM', 'DESCEND', 'DESCRIBE', 'DETERMINE', 'DEVELOPE', 'DIFFER', 'DIMINISH', 'DIRECT', 'DISTRIBUTE', 'DIVIDE', 'CONTRIBUTE', 'DRAIN', 'DRAW', 'SACROCOCCYGEAL_LIGAMENT_ANTERIOR', 'EFFECT', 'ELONGATE', 'EMBRACE', 'ENCIRCLE', 'ENCLOSE', 'END', 'ENSHEATHED_BY', 'ENTER', 'ENVELOP', 'EVERT', 'EXAMINE', 'EXCLUDE', 'EXHIBIT', 'EXIST', 'EXPAND', 'EXTEND', 'FAIL', 'FILL', 'FIT', 'FIX', 'FLARE', 'FLOW', 'FOLLOW', 'ATTACH', 'PROTECT', 'FUNCTION', 'FUSE', 'GIVE', 'GLIDE', 'GROOVE', 'GROUP', 'GROW', 'GUIDE', 'HARMONIZE', 'HAS', 'HAS_ABNORMALITY', 'HAS_ADAPTATION', 'HAS_AGE', 'HAS_ALIAS', 'HAS_ANGLE', 'HAS_APPEARANCE', 'HAS_ARCH', 'HAS_ARRANGEMENT', 'HAS_ARTICULATION', 'HAS_ATTACHMENT', 'HAS_BASE', 'HAS_BONE', 'HAS_CENTER', 'HAS_CIRCUMFERENCE', 'HAS_CONDITION', 'HAS_COUNT', 'HAS_DATE', 'HAS_DEPRESSION', 'HAS_DIAMETER', 'HAS_DISLOCATION', 'HAS_ELASTICITY', 'HAS_ENDS', 'HAS_ESTIMATED_AGE', 'HAS_EXAMPLE', 'HAS_EXCEPTION', 'HAS_FACET', 'HAS_FACETS', 'HAS_FEATURE', 'HAS_FIBERS', 'HAS_FLOOR_OF', 'HAS_FORCE', 'HAS_FUNCTION', 'HAS_FUNCTION_OF', 'HAS_GROOVE', 'HAS_GROWTH', 'HAS_IMPRESSION', 'HAS_INSERTION', 'HAS_JOINT', 'HAS_LAYER', 'HAS_LENGTH', 'HAS_LIGAMENT', 'HAS_LIMITED', 'HAS_LINE', 'HAS_LINING', 'HAS_LOCATION', 'HAS_MAXIMUM', 'HAS_NUCLEI', 'HAS_NUMBER', 'HAS_NUMBER_OF_BONES', 'HAS_ORIGIN', 'HAS_ORIGIN_ON', 'HAS_PLAN', 'HAS_POSITION', 'HAS_PRIMARY_CENTER', 'HAS_PROCESS', 'HAS_PROPERTY', 'HAS_PULL', 'HAS_QUANTITY', 'HAS_RAMIFICATIONS', 'HAS_RESISTANCE', 'HAS_SCALE', 'HAS_SECONDARY_CENTER', 'HAS_SECTION', 'HAS_SEGMENTATION', 'HAS_SEGMENTS', 'HAS_SHAPE', 'HAS_SHEATH', 'HAS_SIMILAR_ARRANGEMENT_TO', 'HAS_SIZE', 'HAS_SMOOTHNESS', 'HAS_SPACES', 'HAS_STAGE', 'HAS_SURFACE', 'HAS_THICKNESS', 'HAS_TYPE', 'HAS_VARIETY', 'HAS_VOLUME', 'HAVE', 'HOLD', 'IMBED', 'IMPLANT', 'IMPORTANT', 'INCREASE', 'INDENT', 'INDICATE', 'INHERIT', 'INSERT', 'INTERPOSE', 'INTERRUPT', 'INTERVENE', 'INVADE', 'INVEST', 'INVOLVE', 'CONTACT', 'IS', 'ACCOMPANY', 'ACCURATE', 'BE_KNOWN_AS', 'APPLY', 'PROLONG', 'BEHIND', 'BLEND', 'BUILD', 'CALL', 'CARRY', 'CHECK', 'BE_CLOSER_TO', 'COMPLETE', 'COMPLICATE', 'CONCAVE', 'BE_CONDITION', 'CONTINUE', 'CONVERT', 'CONVEY', 'COVER', 'DEPRESS', 'DEVELOP', 'DIRECT', 'DIVIDE', 'DRAW', 'EFFECT', 'ELEVATE', 'ENCLOSE', 'ENLARGE', 'BE_EXCEPTION', 'EXPAND', 'EXPEL', 'FILL', 'FIX', 'BE_FOR', 'FORM', 'BE_FREE_FROM', 'FUSE', 'BE_HOMOLOGUE_OF', 'BE_IN', 'INCLUDE', 'INCREASE', 'INDICATE', 'INFRONT_OF', 'BE_INTERMEDIATE', 'INVOLVE', 'LARGER_THAN', 'BE_LIABLE_TO', 'LIMIT', 'LINE', 'LODGE', 'BE_MADE_OF', 'BE_MORE_EXTENSIVE_BETWEEN', 'NARROW', 'CONNECTED_TO', 'NOT_COVERED_BY', 'NOT_RECOGNIZE', 'BE_OF_SIZE', 'PERFORATE', 'PIERCE', 'PLACE', 'BE_POINT_OF', 'BE_POSSIBLE_BY', 'PRESENT_AS', 'PRESS', 'PREVENT', 'PUBLISH', 'RAISE', 'BE_RARE_IN', 'REDUCE', 'REGARD', 'RELAX', 'TENSE', 'RETURN', 'ROOF', 'SEGMENT', 'SHOW', 'BE_SIMILAR_TO', 'BE_SMALLER_THAN', 'STRAIGHTEN', 'STRETCH', 'SUBDIVIDE', 'TERM', 'BE_THICKEST_ALONG', 'TILT', 'BE_TRACE_OF', 'VESTIGIAL', 'MOVE_TRUNK_FORWARD', 'LACK', 'LEAVE', 'LIE', 'LIMIT', 'LINE', 'LIVE_FROM', 'LOCATE', 'LODGE', 'LOOK', 'LOSE', 'LIE_BETWEEN', 'MAGNIFY_AT', 'MAINTAIN', 'MAKE', 'MARK', 'UTILIZE_FOR', 'CONSTITUTE', 'LIE_ON', 'OCCUR_IN', 'MEET_WITH', 'MEET', 'FUSE', 'MENTION', 'MIGRATE_TO', 'MISTRANSLATE_AS', 'MIX', 'MODIFY_TO', 'MOST_COMMON_IN', 'MOVE', 'NOT_PRESENT_IN', 'NUMBER', 'OBLITERATE', 'OCCUPY', 'OCCUR', 'OPEN', 'ORIGIN', 'OSSIFY_FROM', 'OVERCOME', 'OVERLAP', 'OWE', 'PARALLEL_TO', 'INSERT', 'PART_OF', 'PASS', 'PASSAGE', 'PERFORM', 'PERMIT', 'PERSIST', 'PLACE_AT', 'POINT_OUT', 'POSSESS', 'PRECEDE', 'PRESENT', 'PROJECT', 'PROLONG_FROM', 'PROPOSE', 'PROTECT', 'PROTRUDE', 'PROVIDE', 'PUBLISH_IN', 'PUMP_THROUGH', 'RADIATE_FROM', 'RAISE', 'RANGE_FROM', 'REACHE', 'RECEIVE', 'REFLECT', 'REGARD', 'REGULATE', 'RELEASE', 'REPRESENT', 'RESEMBLE', 'RESIST', 'REST_ON', 'RETAIN', 'RETARD', 'RETRACT', 'RETURN_TO', 'RETURN', 'REVOLVE_UPON', 'RISE_FROM', 'ROLL_UPON', 'ROTATE', 'RUDIMENT_OF', 'RUN', 'SEPARATE', 'SHRIVEL', 'SIMILAR_TO', 'SITUATE', 'SPREAD_OVER', 'SPRING_FROM', 'STAIN_WITH', 'START_AT', 'STATE', 'STEADY', 'STRENGTHEN', 'STRETCH', 'SUFFICE_TO_RETAIN', 'SUPPLY', 'SUPPORT', 'SURROUND', 'SUSPEND_FROM', 'SYNCHRONIZE_WITH', 'TAKE_FROM', 'TEND_TO', 'THICKEST_AT', 'TRACE', 'TRANSFER', 'TRANSFORM', 'TRANSMIT', 'TURN_AROUND', 'TYPE_OF', 'UNDERGO', 'UNION_TAKE_PLACE', 'UNITE', 'USE', 'CLOSE_ABOUT', 'VARY_IN', 'VISUALIZE_IN', 'ACCOMPANY', 'AFFPRD_SURFACE_FOR', 'BE_KNOWN_AS', 'ANTAGONIST_OF', 'APPEAR_IN', 'CLOSE_AT', 'COMPOSED', 'DISTRIBUTE', 'BE_FOUND_IN', 'ARISE_FROM', 'ARRANGE', 'ATTACHMENT', 'AUTHOR', 'SEPARATED', 'BEHIND', 'ACT_ON', 'BLEND', 'BLOOD_FROM', 'BRANCH_OF', 'SEPARATE_FROM', 'CAVITY_OF', 'COLLECT', 'COMMUNICATE_WITH', 'COMPLETE', 'COMPOSE', 'COMPRISE', 'CONSTITUTE', 'CONTAIN', 'CONTINUOUS_WITH', 'CONVERGE_TO', 'CONVERT_TO', 'CROSS', 'DEFICIENT_IN', 'DESCEND', 'DESCRIBE_AS', 'DISAPPEAR_UP_TO', 'DISTRIBUTED_TO', 'DRAW', 'ENCIRCLE', 'ENCLOSE', 'END_IN', 'OPEN_INTO', 'ENLARGED', 'ENTER', 'ENTRANCE', 'ESTABLISH', 'EXIST_IN_MIDDLE_OF', 'EXIT', 'EXPANSION_FROM_TENDON', 'FALL_ON', 'FILL', 'FIRST_INDICATION_OF', 'FIX', 'FIND_IN', 'FUSE', 'FUTURE', 'GIVE', 'GROOVED_FOR', 'HAVE', 'HIDE', 'HOLD', 'INDICATE', 'INNERVATE', 'INSERTION', 'INTEGRAL_PART_OF', 'INTERPOSE_BETWEEN', 'INTERRUPT', 'INTERVENE', 'BLEND', 'INVOLVE', 'CONTINUOUS_WITH', 'BE_IN', 'LOCATE', 'COMPOSE', 'NOT_RETURN_TO', 'CONVEY_BY', 'CONVEY_TO', 'SITUATE_BETWEEN', 'JUNCTION_OF', 'LACK', 'LIE', 'LINE', 'LOCATION', 'LODGE', 'LOOSENESS', 'LOWER_BORDER_OF', 'MARK', 'MEASURE', 'MIGRATE_OVER', 'MOVEMENT', 'OCCUPY', 'ONE_FOR', 'OPEN', 'OPPONENT', 'ORIGIN', 'PASS_FORWARD_TO', 'PASSAGE', 'PERFORM', 'PERMIT', 'PLUG', 'PRESENT', 'EXERT_ON', 'PRODUCE', 'PROJECT', 'PROLONGATION_UPWARD_OF', 'PULL', 'PURPOSE', 'PUSH', 'RECEIVE', 'REGULATION', 'RELATION', 'REPRESENT', 'RESIST', 'RETURN_BLOOD_TO', 'RISE', 'ROOF', 'SEPARATE', 'SERVE', 'SITUATE', 'COMPARE_SIZE', 'BE_STRONGEST', 'BE_MOST_DISTINCTSTRUCTURE', 'SUBDIVIDE', 'PREVENT', 'SUPPORT', 'PROTECT', 'SURMOUNT', 'TERMINATE', 'THICKEST_IN', 'THIN', 'TRANSMIT', 'TRAVERSE', 'TRUNK_OF', 'UNDER_SURFACE_OF', 'UNDERGO', 'UNITE', 'VARY', 'VISIBLE']

def extract_entities_and_relations(input_dir, output_dir, pages_to_process=None): 
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    graph_generator = LLMGraphTransformer(
        llm=CustomLLM(generation_loader=generation_loader),
        strict_mode=False, 
        # allowed_nodes=MY_NODE_TYPES,
        # allowed_relationships=MY_RELATIONSHIP_TYPES
    )

    target_files = [
        "1_Embryology.md",
        "2_Osteology.md",
        "3_Syndesmology.md",
        "4_Myology.md",
        "5_Angiology.md"
    ]

    for filename in os.listdir(input_dir):
        if filename in target_files and filename.endswith(".md"):
            filepath = os.path.join(input_dir, filename)
            print(f"Processing {filepath}...")
            with open(filepath, "r", encoding="utf-8") as f:
                content = f.read()

            page_contents = re.findall(r'(#### Page \d+\n.*?)(?=#### Page \d+\n|$)', content, re.DOTALL)
            
            extracted_text = []
            pages_to_iterate = page_contents if pages_to_process is None else page_contents[:pages_to_process]
            
            for i, page_content_raw in enumerate(pages_to_iterate): 
                if not page_content_raw.strip():
                    continue

                try:
                    match = re.search(r'#### Page (\d+)', page_content_raw)
                    page_number = match.group(1) if match else str(i+1)

                    documents = [Document(page_content=page_content_raw, metadata={"page_number": page_number, "source_file": filename})]
                    
                    graph_documents = graph_generator.convert_to_graph_documents(documents)

                    for graph_document in graph_documents:
                        doc_page_number = graph_document.source.metadata.get("page_number", "Unknown")
                        doc_source_file = graph_document.source.metadata.get("source_file", "Unknown")

                        extracted_text.append(f"--- Page {doc_page_number} (File: {doc_source_file}) ---") 
                        extracted_text.append("Entities:")
                        for entity in graph_document.nodes:
                            extracted_text.append(f"  - {entity.type}: {entity.id} (Page: {doc_page_number})")
                        
                        extracted_text.append("\nRelationships:")
                        for relationship in graph_document.relationships:
                            extracted_text.append(f"  - ({relationship.source.id})-[{relationship.type}]->({relationship.target.id}) (Page: {doc_page_number})")
                        extracted_text.append("\n")

                except Exception as e:
                    match = re.search(r'#### Page (\d+)', page_content_raw)
                    page_number = match.group(1) if match else str(i+1)
                    print(f"Error processing page {page_number} of {filename}: {e}")
                    extracted_text.append(f"--- Error processing Page {page_number}: {e} ---")
                
            output_filename = os.path.splitext(filename)[0] + "_extracted_1.txt" ####
            output_filepath = os.path.join(output_dir, output_filename)
            with open(output_filepath, "w", encoding="utf-8") as out_f:
                out_f.write("\n".join(extracted_text))
            print(f"Extraction for {filename} completed. Results saved to {output_filepath}")

class CustomLLM(LLM):
    generation_loader: object
    is_local_model: bool = False
    llm_model: object = None
    llm_tokenizer: object = None

    def __init__(self, generation_loader, **kwargs):
        super().__init__(generation_loader=generation_loader, **kwargs) 
        self.generation_loader = generation_loader
        self.is_local_model = isinstance(self.generation_loader, LocalModelLoader)
        if self.is_local_model:
            self.llm_model = self.generation_loader.model
            self.llm_tokenizer = self.generation_loader.tokenizer

    def _call(self, prompt: str, stop=None) -> str:
        if self.is_local_model:
            inputs = self.llm_tokenizer(prompt, return_tensors="pt").to(self.llm_model.device)
            max_new_tokens = 512

            with torch.no_grad():
                outputs = self.llm_model.generate(
                    **inputs,
                    max_new_tokens = max_new_tokens,
                    num_return_sequences = 1,
                    do_sample = True,
                    temperature = 0.4,
                    top_p = 0.9,
                    repetition_penalty = 1.2,
                    eos_token_id = self.llm_tokenizer.eos_token_id
                )
            generated_text = self.llm_tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
        else:
            generated_text = self.generation_loader.generate(prompt)
        return generated_text

    @property
    def _llm_type(self) -> str:
        return "custom_generation_loader_llm"

if __name__ == "__main__":
    input_directory = "./data/split_file/anatomy"
    output_directory = "./data/extracted_results/LLMGraphTransformer"
    extract_entities_and_relations(input_directory, output_directory, pages_to_process=None)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Processing ./data/split_file/anatomy/1_Embryology.md...
Extraction for 1_Embryology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/1_Embryology_extracted_1.txt
Processing ./data/split_file/anatomy/2_Osteology.md...
Extraction for 2_Osteology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/2_Osteology_extracted_1.txt
Processing ./data/split_file/anatomy/3_Syndesmology.md...
Extraction for 3_Syndesmology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/3_Syndesmology_extracted_1.txt
Processing ./data/split_file/anatomy/4_Myology.md...
Extraction for 4_Myology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/4_Myology_extracted_1.txt
Processing ./data/split_file/anatomy/5_Angiology.md...
Extraction for 5_Angiology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/5_Angiology_extracted_1.txt


: 

# 유일한 개체-관계 추출

In [ ]:
# import os
# import re

# def extract_and_print_unique_types_as_lists(input_dir):
#     unique_entity_types = set()
#     unique_relationship_types = set()

#     relationship_pattern = re.compile(r"\-\[(.+?)\]\->")
#     entity_pattern = re.compile(r"^\s*-\s*([^:]+):")

#     if not os.path.isdir(input_dir):
#         print(f"오류: '{input_dir}' 디렉토리를 찾을 수 없습니다.")
#         return

#     for filename in os.listdir(input_dir):
#         if filename.endswith("_extracted.txt"):
#             filepath = os.path.join(input_dir, filename)
#             with open(filepath, "r", encoding="utf-8") as f:
#                 for line in f:
#                     relationship_match = relationship_pattern.search(line)
#                     if relationship_match:
#                         relationship_type = relationship_match.group(1).strip()
#                         unique_relationship_types.add(relationship_type)
#                     else:
#                         entity_match = entity_pattern.match(line)
#                         if entity_match:
#                             entity_type = entity_match.group(1).strip()
#                             if '(' not in entity_type and ')' not in entity_type:
#                                 unique_entity_types.add(entity_type)

#     sorted_entities = sorted(list(unique_entity_types))
#     sorted_relationships = sorted(list(unique_relationship_types))

#     print("entities =", sorted_entities)
#     print("relations =", sorted_relationships)

# if __name__ == "__main__":
#     input_directory = "./data/extracted_results/LLMGraphTransformer"

#     extract_and_print_unique_types_as_lists(input_directory)


entities = ['Abnormality', 'Action', 'Adjective', 'Age', 'Alias', 'Anatomical Feature', 'Anatomical Foramen', 'Anatomical Group', 'Anatomical Landmark', 'Anatomical Location', 'Anatomical Part', 'Anatomical Process', 'Anatomical Region', 'Anatomical Space', 'Anatomical Structure', 'Anatomical Surface', 'Anatomical Term', 'AnatomicalLocation', 'AnatomicalStructure', 'Anatomical_Structure', 'Anatomy', 'Angle', 'Animal', 'Aponeurosis', 'Area', 'Artery', 'Article', 'Articulation', 'Articulation Type', 'Artifact', 'Author', 'Axis', 'Behavior', 'Biological Entity', 'Biological Material', 'Biological Process', 'Biological Structure', 'Biological Substance', 'Biological Tissue', 'BiologicalEntity', 'BiologicalProcess', 'BiologicalStructure', 'Body Part', 'Body Region', 'BodyFluid', 'BodyPart', 'Body_Part', 'Body_Tissue', 'Body_part', 'Bone', 'Bone Axis', 'Bone Description', 'Bone Feature', 'Bone Part', 'Bone Region', 'Bone Section', 'Bone Structure', 'Bone Surface', 'Bone System', 'Bone Tissue

# 결과물 정제

In [ ]:
# import os
# def add_newline_after_comma(filepath) :
#     try :
#         with open(filepath, 'r', encoding="utf-8") as f :
#             content = f.read()

#         modified_content = content.replace(", ", ",\n")
#         modified_content = content.replace(",", ",\n")

#         with open(filepath, 'w', encoding="utf-8") as f :
#             f.write(modified_content)

#     except FileNotFoundError :
#         print(f"{filepath}를 찾을 수 없음")
#     except Exception as e :
#         print(f"{filepath} 처리 중 오류 발생")

# if __name__ == "__main__" :
#     files_to_process = ["./relations.txt", "./entities.txt"]
#     for file in files_to_process :
#         add_newline_after_comma(file)

# 그래프 구축

In [1]:
import os
import re
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

def build_knowledge_graph(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    G = nx.DiGraph()

    entity_pattern = re.compile(r"\s*-\s*([^:]+):\s*(.*?)\s*\((?:Page|페이지):\s*(\d+)\)")
    relation_pattern = re.compile(r"\s*-\s*\((.*?)\)-\[(.*?)\]->\((.*?)\)\s*\((?:Page|페이지):\s*(\d+)\)")

    for filename in os.listdir(input_dir):
        if filename.endswith("_extracted_1.txt"): ###############
            filepath = os.path.join(input_dir, filename)
            print(f"파일 처리 중: {filepath}")

            with open(filepath, "r", encoding="utf-8") as f:
                content = f.read()

            for match in entity_pattern.finditer(content):
                node_type, node_id, page = match.groups()
                node_id = node_id.strip()
                if not G.has_node(node_id):
                    G.add_node(node_id, type=node_type, source_page=page)
                else:
                    if isinstance(G.nodes[node_id]['source_page'], list):
                        if page not in G.nodes[node_id]['source_page']:
                            G.nodes[node_id]['source_page'].append(page)
                    else:
                        if G.nodes[node_id]['source_page'] != page:
                           G.nodes[node_id]['source_page'] = [G.nodes[node_id]['source_page'], page]


            for match in relation_pattern.finditer(content):
                source_id, rel_type, target_id, page = match.groups()
                source_id = source_id.strip()
                target_id = target_id.strip()

                if G.has_node(source_id) and G.has_node(target_id):
                    G.add_edge(source_id, target_id, type=rel_type, source_page=page)

    print("\n--- 그래프 구축 완료 ---")
    print(f"총 노드 수: {G.number_of_nodes()}")
    print(f"총 엣지 수: {G.number_of_edges()}")

    for node, data in G.nodes(data=True) :
        for key, value in data.items() :
            if isinstance(value, list) :
                G.nodes[node][key] = ','.join(map(str, value))

    for u, v, data in G.edges(data=True) :
        for key, value in data.items() :
            if isinstance(value, list) :
                G.edges[u, v][key] = ','.join(map(str, value))

    graph_output_path = os.path.join(output_dir, "knowledge_graph_1.graphml") ###############
    nx.write_graphml(G, graph_output_path)
    print(f"\n그래프가 {graph_output_path} 에 저장되었습니다.")

    if G.number_of_nodes() > 500:
        print("\n노드 수가 500개를 초과하여 자동 시각화를 건너뜁니다. (너무 오래 걸릴 수 있음)")
        return

    print("그래프 시각화 생성 중...")
    plt.figure(figsize=(20, 20))

    try:
        font_path = "c:/Windows/Fonts/malgun.ttf"  
        font_prop = fm.FontProperties(fname=font_path)
        plt.rcParams['font.family'] = font_prop.get_name()
    except FileNotFoundError:
        print("맑은 고딕 폰트를 찾을 수 없습니다. 기본 폰트로 시각화합니다. (한글이 깨질 수 있음)")

    pos = nx.spring_layout(G, k=0.15, iterations=20, seed=42)
    
    node_labels = {node: node for node in G.nodes()}
    edge_labels = nx.get_edge_attributes(G, 'type')

    nx.draw_networkx_nodes(G, pos, node_size=2000, node_color='skyblue', alpha=0.8)
    nx.draw_networkx_edges(G, pos, edgelist=G.edges(), width=1.5, alpha=0.7, edge_color='gray', arrows=True, arrowsize=20)
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=8, font_family=plt.rcParams['font.family'])
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6, font_color='red')

    plt.title("지식 그래프 시각화", fontsize=20)
    plt.axis('off')
    
    image_output_path = os.path.join(output_dir, "knowledge_graph_1.png") ###############
    plt.savefig(image_output_path, bbox_inches='tight', dpi=300)
    plt.close()

    print(f"그래프 시각화 이미지가 {image_output_path} 에 저장되었습니다.")


if __name__ == "__main__":
    input_directory = "./data/extracted_results/LLMGraphTransformer"
    output_directory = "./data/knowledge_graph"

    build_knowledge_graph(input_directory, output_directory)


파일 처리 중: ./data/extracted_results/LLMGraphTransformer/1_Embryology_extracted_1.txt
파일 처리 중: ./data/extracted_results/LLMGraphTransformer/2_Osteology_extracted_1.txt
파일 처리 중: ./data/extracted_results/LLMGraphTransformer/3_Syndesmology_extracted_1.txt
파일 처리 중: ./data/extracted_results/LLMGraphTransformer/4_Myology_extracted_1.txt
파일 처리 중: ./data/extracted_results/LLMGraphTransformer/5_Angiology_extracted_1.txt

--- 그래프 구축 완료 ---
총 노드 수: 4061
총 엣지 수: 4091

그래프가 ./data/knowledge_graph/knowledge_graph_1.graphml 에 저장되었습니다.

노드 수가 500개를 초과하여 자동 시각화를 건너뜁니다. (너무 오래 걸릴 수 있음)


# 검색

In [7]:
import os
import re
import networkx as nx
import chromadb
import nltk
import json
import nltk.downloader
from nltk.tokenize import sent_tokenize
from typing import List, Dict, Any, Tuple, Optional
from model_loader.config import *
from datetime import datetime

nltk.download("punkt")
nltk.download("punkt_tab")

generation_loader = generation_loader


class QASystem:
    def __init__(self, graphml_path: str, md_path: str, vector_db_path: str = "./chroma_db", similarity_threshold: float = 0.5):
        self.graphml_path = graphml_path
        self.md_path = md_path
        self.similarity_threshold = similarity_threshold
        self.llm_loader = None
        self.kge_model=None
        self.triples_factory=None
        
        self.graph = nx.read_graphml(graphml_path)
        self.client = chromadb.PersistentClient(path=vector_db_path)
        
        self.entity_collection = self.client.get_or_create_collection(name="entities")
        self.relation_collection = self.client.get_or_create_collection(name="relations")
        self.entity_relation_extraction_prompt_template = """
            Extract entities and their relations from the following sentence.

            **Entities** should be **unique nouns or concepts**, extracted as **noun phrases** whenever possible. Identify **concrete objects or concepts** rather than complex activities or phenomena as entities.

            **Relations** should clearly describe the connection between two entities, preferring **reusable predicate verbs** for a knowledge graph. Use **concise verbs** or clear, hyphenated forms like **'part_of' or 'includes'**.

            Output the result **only in the following JSON format**, with no other explanations or text:

            ```json
            {{
                "entities": [
                    {{"name": "Entity1", "type": "Type (e.g., Organ, System, Substance, Function, Disease)"}},
                    {{"name": "Entity2", "type": "Type"}}
                ],
                "relations": [
                    {{"head": "Entity1", "relation": "Relation_Type (e.g., part_of, causes)", "tail": "Entity2"}},
                    {{"head": "Entity3", "relation": "generates", "tail": "Entity4"}}
                ]
            }}

            sentence : "{text_to_analyze}"
            JSON result :
        """

        self._initialize_vector_db()

    def _preprocess_text(self, text: str) -> str:
        return text.upper().replace(' ', '_')

    def _initialize_vector_db(self):
        if self.entity_collection.count() == 0:
            nodes_to_add = []
            unique_nodes = set()
            for node, data in self.graph.nodes(data=True):
                processed_node = self._preprocess_text(node)
                if processed_node not in unique_nodes :
                    metadata = {k: str(v) for k, v in data.items()}
                    metadata['original_name'] = node
                    nodes_to_add.append({'id': processed_node, 'document': processed_node, 'metadata': metadata})
                    unique_nodes.add(processed_node)
            
            if nodes_to_add:
                ids = [item['id'] for item in nodes_to_add]
                documents = [item['document'] for item in nodes_to_add]
                metadatas = [item['metadata'] for item in nodes_to_add]
                self.entity_collection.add(ids=ids, documents=documents, metadatas=metadatas)

        if self.relation_collection.count() == 0:
            edges_to_add = []
            unique_processed_relations = set()
            for u, v, data in self.graph.edges(data=True):
                relation_type = data.get('type')
                if relation_type:
                    processed_relation = self._preprocess_text(relation_type)
                    if processed_relation not in unique_processed_relations :
                        metadata = {'original_name': relation_type}
                        edges_to_add.append({'id': processed_relation, 'document': processed_relation, 'metadata': metadata})
                        unique_processed_relations.add(processed_relation)

            if edges_to_add:
                ids = [item['id'] for item in edges_to_add]
                documents = [item['document'] for item in edges_to_add]
                metadatas = [item['metadata'] for item in edges_to_add]
                self.relation_collection.add(ids=ids, documents=documents, metadatas=metadatas)
    
    def _extract_entities_relations(self, question) :
        prompt = self.entity_relation_extraction_prompt_template.format(text_to_analyze=question)
        raw_llm_output = self._call_llm_generate(prompt)

        try :
            json_start = raw_llm_output.find("{")
            json_end = raw_llm_output.rfind("}") + 1
            if json_start != -1 and json_end != -1 and json_end > json_start :
                json_str = raw_llm_output[json_start:json_end]
                extracted_data = json.loads(json_str)
                return extracted_data.get("entities", []), extracted_data.get("relations", [])
            else :
                print(f"LLM 답변에서 유효한 JSON 형태를 찾을 수 없음 : {raw_llm_output}")
                return [], []
            
        except json.JSONDecodeError as e :
            print(f"개체 추출 과정에서 JSON 디코딩 오류 발생: {e}")
            print(f"오류 발생 원문: {raw_llm_output}")
            return [], []

    def _search_knowledge_graph(self, entities: List[str], relations: List[str]) -> List[Dict[str, Any]]:
        processed_entities = [self._preprocess_text(e["name"]) for e in entities if "name" in e]
        processed_relations = [self._preprocess_text(r["relation"]) for r in relations if "relation" in r]

        found_results = []

        similar_entities = []
        if processed_entities:
            entity_results = self.entity_collection.query(
                query_texts=processed_entities,
                n_results=2, 
                include=["metadatas", "distances"]
            )
            if entity_results['distances']:
                for i, dists in enumerate(entity_results['distances']):
                    for j, dist in enumerate(dists):
                        if dist <= self.similarity_threshold:
                            meta = entity_results['metadatas'][i][j]
                            similar_entities.append(meta['original_name'])
        
        similar_relations = []
        if processed_relations:
            relation_results = self.relation_collection.query(
                query_texts=processed_relations,
                n_results=1,
                include=["metadatas", "distances"]
            )
            if relation_results['distances'] and relation_results['distances'][0]:
                if relation_results['distances'][0][0] <= self.similarity_threshold:
                    meta = relation_results['metadatas'][0][0]
                    similar_relations.append(meta['original_name'])

        similar_entities = list(set(similar_entities))

        inferred_relations = []
        if self.kge_model and self.triples_factory and entities :
            for entity_data in entities :
                entity_name = entity_data["name"]
                head_emb = self._get_kge_embedding(entity_name)
                if head_emb is not None :
                    for rel_name in self.triples_factory.relation_to_id.keys() :
                        rel_emb = self._get_kge_relation_embedding(rel_name)
                        if rel_emb is not None :
                            if rel_name not in similar_relations :
                                inferred_relations.append(rel_name)

        all_relevant_relations = list(set(similar_relations + inferred_relations))

        if similar_entities and all_relevant_relations:
            for u, v, data in self.graph.edges(data=True):
                if (u in similar_entities or v in similar_entities) and data.get('type') in all_relevant_relations:
                    result = data.copy()
                    result['source_node'] = u
                    result['target_node'] = v
                    if 'source_page' in result:
                        found_results.append(result)
        
        if not found_results and similar_entities:
            for u, v, data in self.graph.edges(data=True):
                if u in similar_entities or v in similar_entities:
                    result = data.copy()
                    result['source_node'] = u
                    result['target_node'] = v
                    if 'source_page' in result:
                        found_results.append(result)
        
        if not found_results:
            return []
            
        unique_results = []
        seen = set()
        for res in found_results:
            identifier = (res.get('source_node'), res.get('target_node'), res.get('type'))
            if identifier not in seen:
                unique_results.append(res)
                seen.add(identifier)
                
        return unique_results
    
    def _retrieve_context_from_md(self, search_results: List[Dict[str, Any]], n_sentences: int = 2) -> Tuple[str, List[str]]:
        context = ""
        pages = sorted(list(set(res.get('source_page') for res in search_results if res.get('source_page'))))
        
        if not pages:
            return "", []

        all_md_files = [f for f in os.listdir(self.md_path) if f.endswith('.md')]
        
        page_texts = {}
        for page_num_str in pages:
            page_num = int(page_num_str)
            for md_file in all_md_files:
                with open(os.path.join(self.md_path, md_file), 'r', encoding='utf-8') as f:
                    content = f.read()
                
                match = re.search(rf"####\s+Page\s+{page_num}\b(.*?)(?=####\s+Page|\Z)", content, re.S)
                if match:
                    page_texts[page_num_str] = match.group(1).strip()
                    break
        
        context_parts = []
        for result in search_results:
            page_num = result.get('source_page')
            page_content = page_texts.get(page_num)
            
            if not page_content:
                continue
                
            source_node = result.get('source_node')
            target_node = result.get('target_node')
            
            sentences = sent_tokenize(page_content)
            for i, sent in enumerate(sentences):
                if source_node and source_node in sent and target_node and target_node in sent:
                    start = max(0, i - n_sentences)
                    end = min(len(sentences), i + n_sentences + 1)
                    context_snippet = " ".join(sentences[start:end])
                    context_parts.append(f"... {context_snippet} ... (출처: Page {page_num})")
                    break
        
        context = "\n".join(context_parts)
        return context, pages

    # def _build_llm_prompt(self, question: str, context: str, pages: List[str]) -> str:
    def _build_llm_prompt(self, question: str, context: str) -> str:
        prompt = f"""
        You are a helpful assistant who answers questions based on the provided context.
        You MUST cite the source page number for every piece of information you use.

        **Instructions:**
        1. Answer the user's question clearly and concisely using ONLY the provided context and knowledge graph information.
        2. For every statement, you MUST provide the source page number in parentheses, like this: (Page XX).
        3. If a single piece of information is supported by multiple pages, cite all of them: (Page X, Y, Z).
        4. If no context is available, state that you are answering based on the graph structure alone.

        **Example of a GOOD answer:**
        The ductus arteriosus degenerates into the ligamentum arteriosum after birth(page 360). This is a normal physiological change that happens post-delivery(page 361).

        **Example of a BAD answer:** -> (This is a bad answer because it lacks the mandatory citation)
        The ductus arteriosus becomes the ligamentum arteriosum.

        ---
        **Context:**
        {context}
        ---
        **Question:**
        {question}
        ---
        **Answer:**
        """
        return prompt.strip()
    
    def _call_llm_generate(self, prompt: str) -> str:
        if self.llm_loader:
            if hasattr(self.llm_loader, "tokenizer") and hasattr(self.llm_loader, "model"):
                tokenizer = self.llm_loader.tokenizer
                model = self.llm_loader.model

                input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
                attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)

                output = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=500,
                    temperature=0.0,
                    do_sample=False,
                    top_p=0.85,
                    repetition_penalty=1.2,
                    early_stopping=True,
                    num_beams=3,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )
                generated_ids = output[0][input_ids.shape[-1]:]
                raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
                return raw_answer
            else:
                raw_answer = self.llm_loader.generate(prompt)
                return raw_answer
        else:
            print("generation_loader가 로드되지 않음")
            return "LLM 로더가 설정되지 않았습니다."

    def generate_response(self, question: str) -> Tuple[str, str]:
        entities, relations = self._extract_entities_relations(question)
        if not entities and not relations:
            return "질문에서 유효한 엔티티나 릴레이션을 추출할 수 없습니다.", ""
        
        search_results = self._search_knowledge_graph(entities, relations)
        if not search_results:
            return "관련 정보를 지식 그래프에서 찾을 수 없습니다.", ""
            
        context, pages = self._retrieve_context_from_md(search_results)
        if not context:
            no_context_message = f"관련 정보를 {', '.join(pages) if pages else 'N/A'}에서 발견할 수 없음"
            return no_context_message, ""
        
        prompt = self._build_llm_prompt(question, context)
        
        answer = self._call_llm_generate(prompt)
        return answer, context

def save_results_to_file(question: str, answer: str, context: str, output_dir: str, file_index: int):
    os.makedirs(output_dir, exist_ok=True)

    timestamp = datetime.now().strftime("%H%M%S_%f")
    file_name = f"result_{file_index}_{timestamp}.txt"
    file_path = os.path.join(output_dir, file_name)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(f"[질문]\n{question}\n\n")
        f.write(f"[근거]\n{context}\n\n")
        f.write(f"[답변]\n{answer}\n")

if __name__ == '__main__':
    qa_system = QASystem(
        graphml_path="./data/knowledge_graph/knowledge_graph.graphml",
        md_path="./data/split_file/anatomy/"
    )
    
    qa_system.llm_loader = generation_loader
    
    questions = [
        ############## 1_Embryology.md
        "What are the two essential components of a higher organism cell as defined in the text?", # 7페이지
        "Describe the four main phases of indirect cell division (karyokinesis) as outlined in the text.", # 7페이지
        "What is the primary role of the yolk-sac in the embryo's early development?", # 20페이지
        "How does the embryo separate from the yolk-sac, and what does the enclosed part of the yolk-sac form?", # 19페이지
        "What significant developments occur in a human embryo during the Second Week?", # 33페이지
        "What are the key characteristics of the human embryo by the end of the Third Week?", # 33페이지
        
        ############## 2_Osteology.md
        "What are the three groups into which the cells of a primitive segment differentiate, and what do they form?", # 38페이지
        "How is each vertebral body formed from primitive segments during development?", # 38페이지
        "What are the sphenoidal air sinuses, and where are they located within the sphenoid bone?", # 88페이지
        "Describe the sphenoidal rostrum and its articulation.",# 88
        "What is the tibia, and where is it located in the human leg?", # 158
        "Describe the superior articular surface of the tibia's upper extremity.", # 158

        ############## 3_Syndesmology.md
        "What are joints or articulations, and how are immovable joints characterized?", # 174
        "How does the articular lamella differ from ordinary bone tissue?", # 174
        "Where is the synovial membrane located in relation to the glenoid cavity and humerus, and how does it interact with the Biceps brachii tendon?", # 207
        "List some of the bursae located near the shoulder-joint and specify which ones communicate with the synovial cavity.", # 207
        "What is the function of the plantar calcaneonavicular ligament, and what condition results if it yields?", # 236
        "How are the navicular bone and the three cuneiform bones connected, and what type of movement do they permit?", # 236

        ############## 4_Myology.md
        "How does the nervous system serve as an indicator for the origin and migration paths of developing muscles, despite not influencing muscle differentiation?", # 250
        "Describe the structural components of striped or voluntary muscle, from bundles to individual fibers.", # 250
        "What is the triangular ligament and where is it located?", # 290
        "What structures perforate the superficial layer (inferior fascia) of the urogenital diaphragm?", # 290
        "Where does the Extensor digitorum longus muscle originate, and what structures are located between it and the Tibialis anterior?", # 322
        "What is the Peronæus tertius, and where is it inserted?", # 322

        ############## 5_Angiology.md
        "What are the main characteristics of the middle coat (tunica media) of arteries, and how does its composition vary with vessel size?", # 334
        "Describe the composition and variations of the external coat (tunica adventitia) in arteries.", # 334
        "How do the Vitelline Veins develop into parts of the portal and hepatic veins?", # 345
        "What happens to the Umbilical Veins during embryonic development and after birth?", # 345
        "What are the three phases of a cardiac cycle and what happens during each?", # 358
        "What are the main peculiarities observed in the fetal heart's vascular system?" # 359
    ]   

    today = datetime.now()
    folder_name = f"{today.month}월{today.day}일"
    output_dir = os.path.join("./result", "knowledge_graph", folder_name)
    for i, q in enumerate(questions):
        print(f"질문: {q}")
        response, context = qa_system.generate_response(q)
        print(f"답변: {response}\n")
        save_results_to_file(q, response, context, output_dir, i + 1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


질문: What are the two essential components of a higher organism cell as defined in the text?
답변: The two essential components of a higher organism cell are a soft jelly-like material called cytoplasm and a small spherical body called a nucleus (Page 6).

질문: Describe the four main phases of indirect cell division (karyokinesis) as outlined in the text.
답변: 관련 정보를 지식 그래프에서 찾을 수 없습니다.

질문: What is the primary role of the yolk-sac in the embryo's early development?
답변: According to the text, the primary role of the yolk-sac in the embryo's early development is to provide **nourishment**. It is filled with vitelline fluid which *possibly* utilized for the embryo’s nutrition, and it establishes the **vitelline circulation** – a system where blood conveys nutrients from the yolk-sac to the developing embryo. 

The text also states that a small part of the yolk-sac is enclosed within the embryo and constitutes the primitive digestive tube, further emphasizing its importance in early developmen

# 검색(KGE)

In [9]:
import os
import re
import nltk
import json
import torch
import chromadb
import numpy as np
import networkx as nx
import nltk.downloader
from datetime import datetime
from pykeen.models import ComplEx
from model_loader.config import *
from pykeen.hpo import hpo_pipeline
from pykeen.pipeline import pipeline
from nltk.tokenize import sent_tokenize
from pykeen.optimizers import AdamW as PyKeenAdamW
from typing import List, Dict, Any, Tuple, Optional
from pykeen.triples import TriplesFactory

nltk.download("punkt")
nltk.download("punkt_tab")

generation_loader = generation_loader


class QASystem:
    def __init__(self, graphml_path: str, md_path: str, vector_db_path: str = "./chroma_db", similarity_threshold: float = 0.5):
        self.graphml_path = graphml_path
        self.md_path = md_path
        self.similarity_threshold = similarity_threshold
        self.llm_loader = None
        
        self.graph = nx.read_graphml(graphml_path)
        self.client = chromadb.PersistentClient(path=vector_db_path)
        
        self.entity_collection = self.client.get_or_create_collection(name="entities")
        self.relation_collection = self.client.get_or_create_collection(name="relations")
        self.entity_relation_extraction_prompt_template = """
            Extract entities and their relations from the following sentence.

            **Entities** should be **unique nouns or concepts**, extracted as **noun phrases** whenever possible. Identify **concrete objects or concepts** rather than complex activities or phenomena as entities.

            **Relations** should clearly describe the connection between two entities, preferring **reusable predicate verbs** for a knowledge graph. Use **concise verbs** or clear, hyphenated forms like **'part_of' or 'includes'**.

            Output the result **only in the following JSON format**, with no other explanations or text:

            ```json
            {{
                "entities": [
                    {{"name": "Entity1", "type": "Type (e.g., Organ, System, Substance, Function, Disease)"}},
                    {{"name": "Entity2", "type": "Type"}}
                ],
                "relations": [
                    {{"head": "Entity1", "relation": "Relation_Type (e.g., part_of, causes)", "tail": "Entity2"}},
                    {{"head": "Entity3", "relation": "generates", "tail": "Entity4"}}
                ]
            }}

            sentence : "{text_to_analyze}"
            JSON result :
        """

        self._initialize_vector_db()
        self.kge_model, self.triples_factory = self._train_kge_model()

    def _preprocess_text(self, text: str) -> str:
        return text.upper().replace(' ', '_')

    def _initialize_vector_db(self):
        if self.entity_collection.count() == 0:
            nodes_to_add = []
            unique_nodes = set()
            for node, data in self.graph.nodes(data=True):
                processed_node = self._preprocess_text(node)
                if processed_node not in unique_nodes :
                    metadata = {k: str(v) for k, v in data.items()}
                    metadata['original_name'] = node
                    nodes_to_add.append({'id': processed_node, 'document': processed_node, 'metadata': metadata})
                    unique_nodes.add(processed_node)
            
            if nodes_to_add:
                ids = [item['id'] for item in nodes_to_add]
                documents = [item['document'] for item in nodes_to_add]
                metadatas = [item['metadata'] for item in nodes_to_add]
                self.entity_collection.add(ids=ids, documents=documents, metadatas=metadatas)

        if self.relation_collection.count() == 0:
            edges_to_add = []
            unique_processed_relations = set()
            for u, v, data in self.graph.edges(data=True):
                relation_type = data.get('type')
                if relation_type:
                    processed_relation = self._preprocess_text(relation_type)
                    if processed_relation not in unique_processed_relations :
                        metadata = {'original_name': relation_type}
                        edges_to_add.append({'id': processed_relation, 'document': processed_relation, 'metadata': metadata})
                        unique_processed_relations.add(processed_relation)

            if edges_to_add:
                ids = [item['id'] for item in edges_to_add]
                documents = [item['document'] for item in edges_to_add]
                metadatas = [item['metadata'] for item in edges_to_add]
                self.relation_collection.add(ids=ids, documents=documents, metadatas=metadatas)
    
    def _train_kge_model(self) :
        triples = []
        for u, v, data in self.graph.edges(data=True) :
            relation_type = data.get("type")
            if relation_type and isinstance(relation_type, str):
                triples.append((str(u), str(relation_type), str(v)))

        if not triples :
            print("KGE 모델 학습을 위한 트리플이 없음")
            return None, None
        print(f"생성된 트리플 : {len(triples)}")
        if len(triples) > 0 :
            print(f"첫 5개 : {triples[:5]}")

        triples_array = np.array(triples)
        print(f"Numpy 배열의 형태 : {triples_array.shape}")

        training_triples_factory = TriplesFactory.from_labeled_triples(
            triples=triples_array,
            create_inverse_triples=True
        )
        print(f"TriplesFactory 생성 완료. 엔티티 수 : {training_triples_factory.num_entities}, 관계 수 : {training_triples_factory.num_relations}")

        training_set, testing_set = training_triples_factory.split()

        result = pipeline(
            training=training_set,
            testing=testing_set,
            model=ComplEx,
            optimizer=PyKeenAdamW,
            training_kwargs=dict(num_epochs=100, batch_size=256),
            optimizer_kwargs=dict(lr=0.01),
        )

        print("KGE 모델 학습 완료")
        return result.model, training_triples_factory
    
    def _get_kge_embedding(self, entity_name: str) -> Optional[torch.Tensor] :
        if self.kge_model is None or self.triples_factory is None :
            return None
        
        if entity_name in self.triples_factory.entity_to_id :
            entity_id = self.triples_factory.entity_to_id[entity_name]
            return self.kge_model.entity_representations[0](torch.tensor([entity_id], device=self.kge_model.device)).real.detach().cpu()
        return None
    
    def _get_kge_relation_embedding(self, relation_name: str) -> Optional[torch.Tensor] :
        if self.kge_model is None or self.triples_factory is None :
            return None
        
        if relation_name in self.triples_factory.relation_to_id :
            relation_id = self.triples_factory.relation_to_id[relation_name]
            return self.kge_model.relation_representations[0](torch.tensor([relation_id], device=self.kge_model.device)).real.detach().cpu()
        return None

    def _extract_entities_relations(self, question) :
        prompt = self.entity_relation_extraction_prompt_template.format(text_to_analyze=question)
        raw_llm_output = self._call_llm_generate(prompt)

        try :
            json_start = raw_llm_output.find("{")
            json_end = raw_llm_output.rfind("}") + 1
            if json_start != -1 and json_end != -1 and json_end > json_start :
                json_str = raw_llm_output[json_start:json_end]
                extracted_data = json.loads(json_str)
                return extracted_data.get("entities", []), extracted_data.get("relations", [])
            else :
                print(f"LLM 답변에서 유효한 JSON 형태를 찾을 수 없음 : {raw_llm_output}")
                return [], []
            
        except json.JSONDecodeError as e :
            print(f"개체 추출 과정에서 JSON 디코딩 오류 발생: {e}")
            print(f"오류 발생 원문: {raw_llm_output}")
            return [], []

    def _search_knowledge_graph(self, entities: List[str], relations: List[str]) -> List[Dict[str, Any]]:
        processed_entities = [self._preprocess_text(e["name"]) for e in entities if "name" in e]
        processed_relations = [self._preprocess_text(r["relation"]) for r in relations if "relation" in r]

        found_results = []

        similar_entities = []
        if processed_entities:
            entity_results = self.entity_collection.query(
                query_texts=processed_entities,
                n_results=2, 
                include=["metadatas", "distances"]
            )
            if entity_results['distances']:
                for i, dists in enumerate(entity_results['distances']):
                    for j, dist in enumerate(dists):
                        if dist <= self.similarity_threshold:
                            meta = entity_results['metadatas'][i][j]
                            similar_entities.append(meta['original_name'])
        
        similar_relations = []
        if processed_relations:
            relation_results = self.relation_collection.query(
                query_texts=processed_relations,
                n_results=1,
                include=["metadatas", "distances"]
            )
            if relation_results['distances'] and relation_results['distances'][0]:
                if relation_results['distances'][0][0] <= self.similarity_threshold:
                    meta = relation_results['metadatas'][0][0]
                    similar_relations.append(meta['original_name'])

        similar_entities = list(set(similar_entities))

        inferred_relations = []
        if self.kge_model and entities :
            for entity_data in entities :
                entity_name = entity_data["name"]
                head_emb = self._get_kge_embedding(entity_name)
                if head_emb is not None :
                    for rel_name in self.triples_factory.relation_to_id.keys() :
                        rel_emb = self._get_kge_relation_embedding(rel_name)
                        if rel_emb is not None :
                            if rel_name not in similar_relations :
                                inferred_relations.append(rel_name)

        all_relevant_relations = list(set(similar_relations + inferred_relations))

        if similar_entities and all_relevant_relations:
            for u, v, data in self.graph.edges(data=True):
                if (u in similar_entities or v in similar_entities) and data.get('type') in all_relevant_relations:
                    result = data.copy()
                    result['source_node'] = u
                    result['target_node'] = v
                    if 'source_page' in result:
                        found_results.append(result)
        
        if not found_results and similar_entities:
            for u, v, data in self.graph.edges(data=True):
                if u in similar_entities or v in similar_entities:
                    result = data.copy()
                    result['source_node'] = u
                    result['target_node'] = v
                    if 'source_page' in result:
                        found_results.append(result)
        
        if not found_results:
            return []
            
        unique_results = []
        seen = set()
        for res in found_results:
            identifier = (res.get('source_node'), res.get('target_node'), res.get('type'))
            if identifier not in seen:
                unique_results.append(res)
                seen.add(identifier)
                
        return unique_results
    
    def _retrieve_context_from_md(self, search_results: List[Dict[str, Any]], n_sentences: int = 2) -> Tuple[str, List[str]]:
        context = ""
        pages = sorted(list(set(res.get('source_page') for res in search_results if res.get('source_page'))))
        
        if not pages:
            return "", []

        all_md_files = [f for f in os.listdir(self.md_path) if f.endswith('.md')]
        
        page_texts = {}
        for page_num_str in pages:
            page_num = int(page_num_str)
            for md_file in all_md_files:
                with open(os.path.join(self.md_path, md_file), 'r', encoding='utf-8') as f:
                    content = f.read()
                
                match = re.search(rf"####\s+Page\s+{page_num}\b(.*?)(?=####\s+Page|\Z)", content, re.S)
                if match:
                    page_texts[page_num_str] = match.group(1).strip()
                    break
        
        context_parts = []
        for result in search_results:
            page_num = result.get('source_page')
            page_content = page_texts.get(page_num)
            
            if not page_content:
                continue
                
            source_node = result.get('source_node')
            target_node = result.get('target_node')
            
            sentences = sent_tokenize(page_content)
            for i, sent in enumerate(sentences):
                if source_node and source_node in sent and target_node and target_node in sent:
                    start = max(0, i - n_sentences)
                    end = min(len(sentences), i + n_sentences + 1)
                    context_snippet = " ".join(sentences[start:end])
                    context_parts.append(f"... {context_snippet} ... (출처: Page {page_num})")
                    break
        
        context = "\n".join(context_parts)
        return context, pages

    # def _build_llm_prompt(self, question: str, context: str, pages: List[str]) -> str:
    def _build_llm_prompt(self, question: str, context: str) -> str:
        prompt = f"""
        You are a helpful assistant who answers questions based on the provided context.
        You MUST cite the source page number for every piece of information you use.

        **Instructions:**
        1. Answer the user's question clearly and concisely using ONLY the provided context and knowledge graph information.
        2. For every statement, you MUST provide the source page number in parentheses, like this: (Page XX).
        3. If a single piece of information is supported by multiple pages, cite all of them: (Page X, Y, Z).
        4. If no context is available, state that you are answering based on the graph structure alone.

        **Example of a GOOD answer:**
        The ductus arteriosus degenerates into the ligamentum arteriosum after birth(page 360). This is a normal physiological change that happens post-delivery(page 361).

        **Example of a BAD answer:** -> (This is a bad answer because it lacks the mandatory citation)
        The ductus arteriosus becomes the ligamentum arteriosum.

        ---
        **Context:**
        {context}
        ---
        **Question:**
        {question}
        ---
        **Answer:**
        """
        return prompt.strip()
    
    def _call_llm_generate(self, prompt: str) -> str:
        if self.llm_loader:
            if hasattr(self.llm_loader, "tokenizer") and hasattr(self.llm_loader, "model"):
                tokenizer = self.llm_loader.tokenizer
                model = self.llm_loader.model

                input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
                attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)

                output = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=500,
                    temperature=0.0,
                    do_sample=False,
                    top_p=0.85,
                    repetition_penalty=1.2,
                    early_stopping=True,
                    num_beams=3,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )
                generated_ids = output[0][input_ids.shape[-1]:]
                raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
                return raw_answer
            else:
                raw_answer = self.llm_loader.generate(prompt)
                return raw_answer
        else:
            print("generation_loader가 로드되지 않음")
            return "LLM 로더가 설정되지 않았습니다."

    def generate_response(self, question: str) -> Tuple[str, str]:
        entities, relations = self._extract_entities_relations(question)
        if not entities and not relations:
            return "질문에서 유효한 엔티티나 릴레이션을 추출할 수 없습니다.", ""
        
        search_results = self._search_knowledge_graph(entities, relations)
        if not search_results:
            return "관련 정보를 지식 그래프에서 찾을 수 없습니다.", ""
            
        context, pages = self._retrieve_context_from_md(search_results)
        if not context:
            no_context_message = f"관련 정보를 {', '.join(pages) if pages else 'N/A'}에서 발견할 수 없음"
            return no_context_message, ""
        
        prompt = self._build_llm_prompt(question, context)
        
        answer = self._call_llm_generate(prompt)
        return answer, context

def save_results_to_file(question: str, answer: str, context: str, output_dir: str, file_index: int):
    os.makedirs(output_dir, exist_ok=True)

    timestamp = datetime.now().strftime("%H%M%S_%f")
    file_name = f"result_{file_index}_{timestamp}.txt"
    file_path = os.path.join(output_dir, file_name)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(f"[질문]\n{question}\n\n")
        f.write(f"[근거]\n{context}\n\n")
        f.write(f"[답변]\n{answer}\n")

if __name__ == '__main__':
    qa_system = QASystem(
        graphml_path="./data/knowledge_graph/knowledge_graph.graphml",
        md_path="./data/split_file/anatomy/"
    )
    
    qa_system.llm_loader = generation_loader
    
    questions = [
        ############## 1_Embryology.md
        "What are the two essential components of a higher organism cell as defined in the text?", # 7페이지
        "Describe the four main phases of indirect cell division (karyokinesis) as outlined in the text.", # 7페이지
        "What is the primary role of the yolk-sac in the embryo's early development?", # 20페이지
        "How does the embryo separate from the yolk-sac, and what does the enclosed part of the yolk-sac form?", # 19페이지
        "What significant developments occur in a human embryo during the Second Week?", # 33페이지
        "What are the key characteristics of the human embryo by the end of the Third Week?", # 33페이지
        
        ############## 2_Osteology.md
        "What are the three groups into which the cells of a primitive segment differentiate, and what do they form?", # 38페이지
        "How is each vertebral body formed from primitive segments during development?", # 38페이지
        "What are the sphenoidal air sinuses, and where are they located within the sphenoid bone?", # 88페이지
        "Describe the sphenoidal rostrum and its articulation.",# 88
        "What is the tibia, and where is it located in the human leg?", # 158
        "Describe the superior articular surface of the tibia's upper extremity.", # 158

        ############## 3_Syndesmology.md
        "What are joints or articulations, and how are immovable joints characterized?", # 174
        "How does the articular lamella differ from ordinary bone tissue?", # 174
        "Where is the synovial membrane located in relation to the glenoid cavity and humerus, and how does it interact with the Biceps brachii tendon?", # 207
        "List some of the bursae located near the shoulder-joint and specify which ones communicate with the synovial cavity.", # 207
        "What is the function of the plantar calcaneonavicular ligament, and what condition results if it yields?", # 236
        "How are the navicular bone and the three cuneiform bones connected, and what type of movement do they permit?", # 236

        ############## 4_Myology.md
        "How does the nervous system serve as an indicator for the origin and migration paths of developing muscles, despite not influencing muscle differentiation?", # 250
        "Describe the structural components of striped or voluntary muscle, from bundles to individual fibers.", # 250
        "What is the triangular ligament and where is it located?", # 290
        "What structures perforate the superficial layer (inferior fascia) of the urogenital diaphragm?", # 290
        "Where does the Extensor digitorum longus muscle originate, and what structures are located between it and the Tibialis anterior?", # 322
        "What is the Peronæus tertius, and where is it inserted?", # 322

        ############## 5_Angiology.md
        "What are the main characteristics of the middle coat (tunica media) of arteries, and how does its composition vary with vessel size?", # 334
        "Describe the composition and variations of the external coat (tunica adventitia) in arteries.", # 334
        "How do the Vitelline Veins develop into parts of the portal and hepatic veins?", # 345
        "What happens to the Umbilical Veins during embryonic development and after birth?", # 345
        "What are the three phases of a cardiac cycle and what happens during each?", # 358
        "What are the main peculiarities observed in the fetal heart's vascular system?" # 359
    ]   

    today = datetime.now()
    folder_name = f"{today.month}월{today.day}일"
    output_dir = os.path.join("./result", "knowledge_graph", folder_name)
    for i, q in enumerate(questions):
        print(f"질문: {q}")
        response, context = qa_system.generate_response(q)
        print(f"답변: {response}\n")
        save_results_to_file(q, response, context, output_dir, i + 1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [423, 827]
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.triples.triples_factory:Creating inverse triples.


생성된 트리플 : 4132
첫 5개 : [('germinal cells', 'LOCATED_IN', 'ovaries'), ('granules', 'STAIN_WITH', 'eosin'), ('nucleus', 'IS_SURROUNDED_BY', 'protoplasm'), ('nucleus', 'HAS_COUNT', 'twenty-four'), ('hyaloplasm', 'IS_PART_OF', 'cytoplasm')]
Numpy 배열의 형태 : (4132, 3)
TriplesFactory 생성 완료. 엔티티 수 : 3756, 관계 수 : 2148


Training epochs on cuda:0:   0%|          | 0/100 [00:00<?, ?epoch/s]

INFO:pykeen.triples.triples_factory:Creating inverse triples.


Training batches on cuda:0:   0%|          | 0.00/26.0 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/26.0 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/827 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.05s seconds


KGE 모델 학습 완료
질문: What are the two essential components of a higher organism cell as defined in the text?
답변: The two essentials of a cell in higher organisms are a soft jelly-like material called cytoplasm and a small spherical body called a nucleus (Page 6).

질문: Describe the four main phases of indirect cell division (karyokinesis) as outlined in the text.
답변: 관련 정보를 지식 그래프에서 찾을 수 없습니다.

질문: What is the primary role of the yolk-sac in the embryo's early development?
답변: According to the text, the primary role of the yolk-sac in the embryo's early development is to provide **nourishment**. 

Specifically, the text states that the yolk-sac is filled with **vitelline fluid** which "possibly may be utilized for the nourishment of the embryo during the earlier stages of its existence." It also describes **vitelline circulation** where blood conveys nutrients from the yolk-sac to the embryo's heart. 

While it initially forms part of the digestive tube and later reduces in size, its core f

: 

In [ ]:
# import os
# import torch
# from langchain.schema import Generation, LLMResult
# from langchain.llms.base import LLM
# from model_loader.config import generation_loader
# from langchain_core.documents import Document
# from model_loader.local_loader import LocalModelLoader
# from langchain_experimental.graph_transformers import LLMGraphTransformer

# def extract_entities_and_relations(input_dir, output_dir, pages_to_process=5):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     graph_generator = LLMGraphTransformer(llm=CustomLLM(generation_loader=generation_loader), strict_mode=False)

#     for filename in os.listdir(input_dir):
#         if filename.endswith(".md"):
#             filepath = os.path.join(input_dir, filename)
#             print(f"Processing {filepath}...")
#             with open(filepath, "r", encoding="utf-8") as f:
#                 content = f.read()

#             pages = content.split('\n\n') 
            
#             extracted_text = []
#             for i, page_content in enumerate(pages[:pages_to_process]):
#                 if not page_content.strip():
#                     continue

#                 try:
#                     documents = [Document(page_content=page_content)]
                    
#                     graph_documents = graph_generator.convert_to_graph_documents(documents)

#                     for graph_document in graph_documents:
#                         extracted_text.append(f"--- Page {i+1} ---")
#                         extracted_text.append("Entities:")
#                         for entity in graph_document.nodes:
#                             extracted_text.append(f"  - {entity.type}: {entity.id}")
                        
#                         extracted_text.append("\nRelationships:")
#                         for relationship in graph_document.relationships:
#                             extracted_text.append(f"  - ({relationship.source.id})-[{relationship.type}]->({relationship.target.id})")
#                         extracted_text.append("\n")

#                 except Exception as e:
#                     print(f"Error processing page {i+1} of {filename}: {e}")
#                     extracted_text.append(f"--- Error processing Page {i+1}: {e} ---")
                
#             output_filename = os.path.splitext(filename)[0] + "_extracted.txt"
#             output_filepath = os.path.join(output_dir, output_filename)
#             with open(output_filepath, "w", encoding="utf-8") as out_f:
#                 out_f.write("\n".join(extracted_text))
#             print(f"Extraction for {filename} completed. Results saved to {output_filepath}")

# class CustomLLM(LLM):
#     generation_loader: object
#     is_local_model: bool = False
#     llm_model: object = None
#     llm_tokenizer: object = None

#     def __init__(self, generation_loader, **kwargs):
#         super().__init__(generation_loader=generation_loader, **kwargs) 
#         self.generation_loader = generation_loader
#         self.is_local_model = isinstance(self.generation_loader, LocalModelLoader)
#         if self.is_local_model:
#             self.llm_model = self.generation_loader.model
#             self.llm_tokenizer = self.generation_loader.tokenizer

#     def _call(self, prompt: str, stop=None) -> str:
#         if self.is_local_model:
#             inputs = self.llm_tokenizer(prompt, return_tensors="pt").to(self.llm_model.device)
#             max_new_tokens = 512

#             with torch.no_grad():
#                 outputs = self.llm_model.generate(
#                     **inputs,
#                     max_new_tokens = max_new_tokens,
#                     num_return_sequences = 1,
#                     do_sample = True,
#                     temperature = 0.4,
#                     top_p = 0.9,
#                     repetition_penalty = 1.2,
#                     eos_token_id = self.llm_tokenizer.eos_token_id
#                 )
#             generated_text = self.llm_tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
#         else:
#             generated_text = self.generation_loader.generate(prompt)
#         return generated_text

#     @property
#     def _llm_type(self) -> str:
#         return "custom_generation_loader_llm"

# if __name__ == "__main__":
#     input_directory = "./data/split_file/anatomy"
#     output_directory = "./data/extracted_results/LLMGraphTransformer"
#     extract_entities_and_relations(input_directory, output_directory, pages_to_process=5)


Processing ./data/split_file/anatomy/1_Embryology.md...
Extraction for 1_Embryology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/1_Embryology_extracted.txt
Processing ./data/split_file/anatomy/2_Osteology.md...
Extraction for 2_Osteology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/2_Osteology_extracted.txt
Processing ./data/split_file/anatomy/3_Syndesmology.md...
Extraction for 3_Syndesmology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/3_Syndesmology_extracted.txt
Processing ./data/split_file/anatomy/4_Myology.md...
Extraction for 4_Myology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/4_Myology_extracted.txt
Processing ./data/split_file/anatomy/5_Angiology.md...
Extraction for 5_Angiology.md completed. Results saved to ./data/extracted_results/LLMGraphTransformer/5_Angiology_extracted.txt
Processing ./data/split_file/anatomy/9_Neurology.md...
Extraction for 9_Ne

: 

In [ ]:
MY_NODE_TYPES = ['Adjective', 'Alias', 'Anatomical Foramen', 'Anatomical Group', 'Anatomical Landmark', 'Anatomical Process', 'AnatomicalRegion', 'AnatomicalSpace', 'Anatomical Surface', 'Anatomical Term', 'AnatomicalStructure', 'Anatomy', 'Animal', 'Aponeurosis', 'Artifact', 'Axis', 'Biological Material', 'Biological Process', 'BiologicalEntity', 'BiologicalProcess', 'BodyFluid', 'Bone', 'Bone Axis', 'Bone Description', 'Bone System', 'Bone Tissue', 'Bone Type', 'Border', 'Boundary', 'Brain Structure', 'Brainstem', 'Canal', 'Capsule', 'Cartilage', 'Cell', 'Cell Type', 'CellStructure', 'CellType', 'Characteristic', 'Chemical', 'Circle', 'Circulation', 'Circulatory System', 'Crest', 'Curve', 'Data', 'Date', 'Depression', 'Description', 'Direction', 'Disk', 'Diverticulum', 'Duct', 'Dye', 'Element', 'Eminence', 'Environment', 'Event', 'Extremity', 'Fascia', 'Fiber', 'Field of Study', 'Fluid', 'Foramen', 'Formation', 'Gender', 'GeometricProperty', 'Gland', 'Groove', 'Group', 'Information', 'Joint', 'Junction', 'Law', 'Layer', 'Life Stage', 'Ligament', 'Line', 'Manner', 'MaterialProperty', 'Meatus', 'Mechanism', 'MedicalCondition', 'Membrane', 'Molecule', 'Muscle', 'Muscle Action', 'Nerve', 'Nervous System', 'Network', 'Node', 'Nomenclature', 'Notch', 'Observation', 'Organ', 'Organ System', 'Organelle', 'Organism', 'Origin', 'Path', 'Person', 'Physical Contact', 'PhysicalAction', 'PhysicalProperty', 'PhysicalSupport', 'Plane', 'Plate', 'Point', 'Process', 'Profession', 'Property', 'Protein', 'Publication', 'Quantity', 'Ridge', 'Section', 'Septum', 'Shape', 'Sinus', 'Skin', 'SpatialArrangement', 'Spine', 'State', 'Substance', 'Surface', 'System', 'Tendency', 'Tendon', 'Theory', 'Tissue', 'Tool', 'Tooth', 'Topic', 'Tuberosity', 'Valve', 'Vesicle', 'Vessel', 'Work', 'anatomy', 'body_part', 'ligament', 'nerve']

MY_RELATIONSHIP_TYPES = ['ABDUCT', 'ABSORB', 'ACCOMPLISH', 'ACCUMULATE', 'ACTION', 'ADAPT', 'ADMIT', 'AFFECT', 'ALLOW', 'ANTAGONIZE', 'APPEAR', 'ARCHES_OVER', 'ARCH_FORWARD', 'ARE', 'ATTACH', 'BRING', 'CHARACTERIZE', 'ESTABLISH', 'EXPAND', 'FORM', 'FUSE', 'MAKE', 'OCCUPY', 'ORIFICE', 'OSSIFY', 'REPLACE', 'ARISE', 'ARRANGE', 'ARTICULATE', 'ASSIST', 'ASSUME', 'ATTACHMENT', 'AUTHORED', 'AUTHOR_OF', 'SACROCOCCYGEAL_LIGAMENT_POSTERIOR', 'BEAR', 'BEGIN', 'BELIEVE', 'BELOW', 'BEND', 'BIND', 'BLEND', 'BOUND', 'BREAK', 'BRIDGE', 'DO', 'CAPABLE_OF', 'CARRY', 'CAUSE', 'CHANGE', 'CHECK', 'CLOSE', 'CLOTH', 'COME', 'COMMENCE', 'COMMUNICATE', 'COMPLETE', 'COMPOSE', 'COMPRESS', 'CONNECT', 'CONSTITUTE', 'CONTINUE', 'CONTRACT', 'CONTRIBUTE', 'CONVERGE', 'CONVERT', 'CORRESPOND', 'COVER', 'CREATE', 'CROSS', 'CURVE', 'CUT', 'SACROCOCCYGEAL_LIGAMENT_LATERAL', 'PUBIC_ARCH_FORMATION', 'SUPERIOR_PUBIC_LIGAMENT', 'DECREASE', 'DEEPEN', 'DEGENERATE', 'DEPOSIT', 'DERIVE_FROM', 'DESCEND', 'DESCRIBE', 'DETERMINE', 'DEVELOPE', 'DIFFER', 'DIMINISH', 'DIRECT', 'DISTRIBUTE', 'DIVIDE', 'CONTRIBUTE', 'DRAIN', 'DRAW', 'SACROCOCCYGEAL_LIGAMENT_ANTERIOR', 'EFFECT', 'ELONGATE', 'EMBRACE', 'ENCIRCLE', 'ENCLOSE', 'END', 'ENSHEATHED_BY', 'ENTER', 'ENVELOP', 'EVERT', 'EXAMINE', 'EXCLUDE', 'EXHIBIT', 'EXIST', 'EXPAND', 'EXTEND', 'FAIL', 'FILL', 'FIT', 'FIX', 'FLARE', 'FLOW', 'FOLLOW', 'ATTACH', 'PROTECT', 'FUNCTION', 'FUSE', 'GIVE', 'GLIDE', 'GROOVE', 'GROUP', 'GROW', 'GUIDE', 'HARMONIZE', 'HAS', 'HAS_ABNORMALITY', 'HAS_ADAPTATION', 'HAS_AGE', 'HAS_ALIAS', 'HAS_ANGLE', 'HAS_APPEARANCE', 'HAS_ARCH', 'HAS_ARRANGEMENT', 'HAS_ARTICULATION', 'HAS_ATTACHMENT', 'HAS_BASE', 'HAS_BONE', 'HAS_CENTER', 'HAS_CIRCUMFERENCE', 'HAS_CONDITION', 'HAS_COUNT', 'HAS_DATE', 'HAS_DEPRESSION', 'HAS_DIAMETER', 'HAS_DISLOCATION', 'HAS_ELASTICITY', 'HAS_ENDS', 'HAS_ESTIMATED_AGE', 'HAS_EXAMPLE', 'HAS_EXCEPTION', 'HAS_FACET', 'HAS_FACETS', 'HAS_FEATURE', 'HAS_FIBERS', 'HAS_FLOOR_OF', 'HAS_FORCE', 'HAS_FUNCTION', 'HAS_FUNCTION_OF', 'HAS_GROOVE', 'HAS_GROWTH', 'HAS_IMPRESSION', 'HAS_INSERTION', 'HAS_JOINT', 'HAS_LAYER', 'HAS_LENGTH', 'HAS_LIGAMENT', 'HAS_LIMITED', 'HAS_LINE', 'HAS_LINING', 'HAS_LOCATION', 'HAS_MAXIMUM', 'HAS_NUCLEI', 'HAS_NUMBER', 'HAS_NUMBER_OF_BONES', 'HAS_ORIGIN', 'HAS_ORIGIN_ON', 'HAS_PLAN', 'HAS_POSITION', 'HAS_PRIMARY_CENTER', 'HAS_PROCESS', 'HAS_PROPERTY', 'HAS_PULL', 'HAS_QUANTITY', 'HAS_RAMIFICATIONS', 'HAS_RESISTANCE', 'HAS_SCALE', 'HAS_SECONDARY_CENTER', 'HAS_SECTION', 'HAS_SEGMENTATION', 'HAS_SEGMENTS', 'HAS_SHAPE', 'HAS_SHEATH', 'HAS_SIMILAR_ARRANGEMENT_TO', 'HAS_SIZE', 'HAS_SMOOTHNESS', 'HAS_SPACES', 'HAS_STAGE', 'HAS_SURFACE', 'HAS_THICKNESS', 'HAS_TYPE', 'HAS_VARIETY', 'HAS_VOLUME', 'HAVE', 'HOLD', 'IMBED', 'IMPLANT', 'IMPORTANT', 'INCREASE', 'INDENT', 'INDICATE', 'INHERIT', 'INSERT', 'INTERPOSE', 'INTERRUPT', 'INTERVENE', 'INVADE', 'INVEST', 'INVOLVE', 'CONTACT', 'IS', 'ACCOMPANY', 'ACCURATE', 'BE_KNOWN_AS', 'APPLY', 'PROLONG', 'BEHIND', 'BLEND', 'BUILD', 'CALL', 'CARRY', 'CHECK', 'BE_CLOSER_TO', 'COMPLETE', 'COMPLICATE', 'CONCAVE', 'BE_CONDITION', 'CONTINUE', 'CONVERT', 'CONVEY', 'COVER', 'DEPRESS', 'DEVELOP', 'DIRECT', 'DIVIDE', 'DRAW', 'EFFECT', 'ELEVATE', 'ENCLOSE', 'ENLARGE', 'BE_EXCEPTION', 'EXPAND', 'EXPEL', 'FILL', 'FIX', 'BE_FOR', 'FORM', 'BE_FREE_FROM', 'FUSE', 'BE_HOMOLOGUE_OF', 'BE_IN', 'INCLUDE', 'INCREASE', 'INDICATE', 'INFRONT_OF', 'BE_INTERMEDIATE', 'INVOLVE', 'LARGER_THAN', 'BE_LIABLE_TO', 'LIMIT', 'LINE', 'LODGE', 'BE_MADE_OF', 'BE_MORE_EXTENSIVE_BETWEEN', 'NARROW', 'CONNECTED_TO', 'NOT_COVERED_BY', 'NOT_RECOGNIZE', 'BE_OF_SIZE', 'PERFORATE', 'PIERCE', 'PLACE', 'BE_POINT_OF', 'BE_POSSIBLE_BY', 'PRESENT_AS', 'PRESS', 'PREVENT', 'PUBLISH', 'RAISE', 'BE_RARE_IN', 'REDUCE', 'REGARD', 'RELAX', 'TENSE', 'RETURN', 'ROOF', 'SEGMENT', 'SHOW', 'BE_SIMILAR_TO', 'BE_SMALLER_THAN', 'STRAIGHTEN', 'STRETCH', 'SUBDIVIDE', 'TERM', 'BE_THICKEST_ALONG', 'TILT', 'BE_TRACE_OF', 'VESTIGIAL', 'MOVE_TRUNK_FORWARD', 'LACK', 'LEAVE', 'LIE', 'LIMIT', 'LINE', 'LIVE_FROM', 'LOCATE', 'LODGE', 'LOOK', 'LOSE', 'LIE_BETWEEN', 'MAGNIFY_AT', 'MAINTAIN', 'MAKE', 'MARK', 'UTILIZE_FOR', 'CONSTITUTE', 'LIE_ON', 'OCCUR_IN', 'MEET_WITH', 'MEET', 'FUSE', 'MENTION', 'MIGRATE_TO', 'MISTRANSLATE_AS', 'MIX', 'MODIFY_TO', 'MOST_COMMON_IN', 'MOVE', 'NOT_PRESENT_IN', 'NUMBER', 'OBLITERATE', 'OCCUPY', 'OCCUR', 'OPEN', 'ORIGIN', 'OSSIFY_FROM', 'OVERCOME', 'OVERLAP', 'OWE', 'PARALLEL_TO', 'INSERT', 'PART_OF', 'PASS', 'PASSAGE', 'PERFORM', 'PERMIT', 'PERSIST', 'PLACE_AT', 'POINT_OUT', 'POSSESS', 'PRECEDE', 'PRESENT', 'PROJECT', 'PROLONG_FROM', 'PROPOSE', 'PROTECT', 'PROTRUDE', 'PROVIDE', 'PUBLISH_IN', 'PUMP_THROUGH', 'RADIATE_FROM', 'RAISE', 'RANGE_FROM', 'REACHE', 'RECEIVE', 'REFLECT', 'REGARD', 'REGULATE', 'RELEASE', 'REPRESENT', 'RESEMBLE', 'RESIST', 'REST_ON', 'RETAIN', 'RETARD', 'RETRACT', 'RETURN_TO', 'RETURN', 'REVOLVE_UPON', 'RISE_FROM', 'ROLL_UPON', 'ROTATE', 'RUDIMENT_OF', 'RUN', 'SEPARATE', 'SHRIVEL', 'SIMILAR_TO', 'SITUATE', 'SPREAD_OVER', 'SPRING_FROM', 'STAIN_WITH', 'START_AT', 'STATE', 'STEADY', 'STRENGTHEN', 'STRETCH', 'SUFFICE_TO_RETAIN', 'SUPPLY', 'SUPPORT', 'SURROUND', 'SUSPEND_FROM', 'SYNCHRONIZE_WITH', 'TAKE_FROM', 'TEND_TO', 'THICKEST_AT', 'TRACE', 'TRANSFER', 'TRANSFORM', 'TRANSMIT', 'TURN_AROUND', 'TYPE_OF', 'UNDERGO', 'UNION_TAKE_PLACE', 'UNITE', 'USE', 'CLOSE_ABOUT', 'VARY_IN', 'VISUALIZE_IN', 'ACCOMPANY', 'AFFPRD_SURFACE_FOR', 'be known as', 'antagonist of', 'appear in', 'close at', 'composed', 'distribute', 'be found in', 'arise from', 'arrange', 'attachment', 'author', 'separated', 'behind', 'act on', 'blend', 'blood from', 'branch of', 'separate from', 'cavity of', 'collect', 'communicate with', 'complete', 'compose', 'comprise', 'constitute', 'contain', 'continuous with', 'converge to', 'convert_to', 'cross', 'deficient in', 'descend', 'describe as', 'disappear up to', 'distributed to', 'draw', 'encircle', 'enclose', 'end in', 'open into', 'enlarged', 'enter', 'entrance', 'establish', 'exist in middle of', 'exit', 'expansion from tendon', 'fall on', 'fill', 'first indication of', 'fix', 'find in', 'fuse', 'future', 'give', 'grooved for', 'have', 'hide', 'hold', 'indicate', 'innervate', 'insertion', 'integral part of', 'interpose between', 'interrupt', 'intervene', 'blend', 'involve', 'continuous with', 'be in', 'locate', 'compose', 'not return to', 'convey by', 'convey to', 'situate between', 'junction of', 'lack', 'lie', 'line', 'location', 'lodge', 'looseness', 'lower border of', 'mark', 'measure', 'migrate over', 'movement', 'occupy', 'one for', 'open', 'opponent', 'origin', 'pass forward to', 'passage', 'perform', 'permit', 'plug', 'present', 'exert on', 'produce', 'project', 'prolongation upward of', 'pull', 'purpose', 'push', 'receive', 'regulation', 'relation', 'represent', 'resist', 'return blood to', 'rise', 'roof', 'separate', 'serve', 'situate', 'compare size', 'be strongest', 'be most distinct' 'structure', 'subdivide', 'prevent', 'support', 'protect', 'surmount', 'terminate', 'thickest in', 'thin', 'transmit', 'traverse', 'trunk of', 'under surface of', 'undergo', 'unite', 'vary', 'visible']

output_entity = [s.upper().replace(' ', '_') for s in MY_NODE_TYPES]
output_relation = [s.upper().replace(' ', '_') for s in MY_RELATIONSHIP_TYPES]
print("MY_NODE_TYPES : ")
print(output_entity)
print(f"entity 개수 : {len(output_entity)}")

print("MY_RELATIONSHIP_TYPES : ")
print(output_relation)
print(f"relation 개수 : {len(output_relation)}")

MY_NODE_TYPES : 
['ADJECTIVE', 'ALIAS', 'ANATOMICAL_FORAMEN', 'ANATOMICAL_GROUP', 'ANATOMICAL_LANDMARK', 'ANATOMICAL_PROCESS', 'ANATOMICALREGION', 'ANATOMICALSPACE', 'ANATOMICAL_SURFACE', 'ANATOMICAL_TERM', 'ANATOMICALSTRUCTURE', 'ANATOMY', 'ANIMAL', 'APONEUROSIS', 'ARTIFACT', 'AXIS', 'BIOLOGICAL_MATERIAL', 'BIOLOGICAL_PROCESS', 'BIOLOGICALENTITY', 'BIOLOGICALPROCESS', 'BODYFLUID', 'BONE', 'BONE_AXIS', 'BONE_DESCRIPTION', 'BONE_SYSTEM', 'BONE_TISSUE', 'BONE_TYPE', 'BORDER', 'BOUNDARY', 'BRAIN_STRUCTURE', 'BRAINSTEM', 'CANAL', 'CAPSULE', 'CARTILAGE', 'CELL', 'CELL_TYPE', 'CELLSTRUCTURE', 'CELLTYPE', 'CHARACTERISTIC', 'CHEMICAL', 'CIRCLE', 'CIRCULATION', 'CIRCULATORY_SYSTEM', 'CREST', 'CURVE', 'DATA', 'DATE', 'DEPRESSION', 'DESCRIPTION', 'DIRECTION', 'DISK', 'DIVERTICULUM', 'DUCT', 'DYE', 'ELEMENT', 'EMINENCE', 'ENVIRONMENT', 'EVENT', 'EXTREMITY', 'FASCIA', 'FIBER', 'FIELD_OF_STUDY', 'FLUID', 'FORAMEN', 'FORMATION', 'GENDER', 'GEOMETRICPROPERTY', 'GLAND', 'GROOVE', 'GROUP', 'INFORMATION'

: 

In [6]:
import os
import glob

def calculate_overall_average_paragraph_length(directory_path: str) -> None:
    if not os.path.isdir(directory_path):
        print(f"오류: '{directory_path}' 디렉터리를 찾을 수 없습니다.")
        return

    md_files = glob.glob(os.path.join(directory_path, '*.md'))

    if not md_files:
        print(f"오류: '{directory_path}' 디렉터리에서 .md 파일을 찾을 수 없습니다.")
        return

    total_paragraph_length = 0
    total_paragraph_count = 0

    for file_path in md_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            paragraphs = content.split('\n\n')
            
            for p in paragraphs:
                stripped_p = p.strip()
                if stripped_p:
                    total_paragraph_length += len(stripped_p)
                    total_paragraph_count += 1
        except Exception as e:
            print(f"'{file_path}' 파일을 처리하는 중 오류 발생: {e}")

    if total_paragraph_count == 0:
        print("분석할 문단이 없습니다.")
        return

    average_length = total_paragraph_length / total_paragraph_count

    print(f"분석 대상 폴더: {directory_path}")
    print(f"분석된 .md 파일 개수: {len(md_files)}개")
    print("-" * 30)
    print(f"전체 문단 개수: {total_paragraph_count}개")
    print(f"전체 문단 글자 수 합계: {total_paragraph_length}자")
    print(f"모든 파일의 문단 평균 길이: {average_length:.2f}자")

if __name__ == '__main__':
    anatomy_folder_path = './data/split_file/anatomy/'
    calculate_overall_average_paragraph_length(anatomy_folder_path)

분석 대상 폴더: ./data/split_file/anatomy/
분석된 .md 파일 개수: 12개
------------------------------
전체 문단 개수: 9083개
전체 문단 글자 수 합계: 3665997자
모든 파일의 문단 평균 길이: 403.61자


In [ ]:
import os
import glob

def find_max_paragraph_length_per_file(directory_path: str) -> None:
    if not os.path.isdir(directory_path):
        print(f"오류: '{directory_path}' 디렉터리를 찾을 수 없습니다.")
        return

    md_files = glob.glob(os.path.join(directory_path, '*.md'))

    if not md_files:
        print(f"오류: '{directory_path}' 디렉터리에서 .md 파일을 찾을 수 없습니다.")
        return
        
    print(f"분석 대상 폴더: {directory_path}")
    print("-" * 40)

    for file_path in md_files:
        max_length_in_file = 0
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            paragraphs = content.split('\n\n')
            
            for p in paragraphs:
                stripped_p = p.strip()
                if stripped_p:
                    if len(stripped_p) > max_length_in_file:
                        max_length_in_file = len(stripped_p)
            
            file_name = os.path.basename(file_path)
            print(f"📄 파일: {file_name:<20} | 최대 문단 길이: {max_length_in_file}자")

        except Exception as e:
            print(f"'{file_path}' 파일을 처리하는 중 오류 발생: {e}")

if __name__ == '__main__':
    anatomy_folder_path = './data/split_file/anatomy/'
    find_max_paragraph_length_per_file(anatomy_folder_path)

분석 대상 폴더: ./data/split_file/anatomy/
----------------------------------------
📄 파일: 1_Embryology.md      | 최대 문단 길이: 3755자
📄 파일: 2_Osteology.md       | 최대 문단 길이: 5589자
📄 파일: 3_Syndesmology.md    | 최대 문단 길이: 3140자
📄 파일: 4_Myology.md         | 최대 문단 길이: 3162자
📄 파일: 5_Angiology.md       | 최대 문단 길이: 2970자
📄 파일: 9_Neurology.md       | 최대 문단 길이: 2476자
📄 파일: 11_Splanchnology.md  | 최대 문단 길이: 4685자
📄 파일: 6_The_Arteries.md    | 최대 문단 길이: 2412자
📄 파일: 7_The_Veins.md       | 최대 문단 길이: 1745자
📄 파일: 8_The_Lymphatic_System.md | 최대 문단 길이: 2908자
📄 파일: 10_The_Organs_of_the_Senses_and_the_Common_Integument.md | 최대 문단 길이: 2612자
📄 파일: 12_Surface_Anatomy_and_Surface_Markings.md | 최대 문단 길이: 3148자


In [ ]:
import os
import json

def count(directory_path) :
    total_qa_count = 0

    if not os.path.isdir(directory_path) :
        print(f"{directory_path}를 찾을 수 없습니다")
        return 0
    
    for filename in os.listdir(directory_path) :
        if filename.endswith(".json") :
            file_path = os.path.join(directory_path, filename)

            try :
                with open(file_path, 'r', encoding="utf-8") as f :
                    data = json.load(f)
                    if isinstance(data, list) :
                        total_qa_count += len(data)
                    elif isinstance(data, dict) :
                        total_qa_count += 1
            except json.JSONDecodeError :
                print(f"{file_path} 파일의 형식이 잘못됨")
            except Exception as e :
                print(f"{file_path} 파일을 처리하는 중 오류 : {e}")

    return total_qa_count

target_directory = "./data/HarryPotterQA"

total_pairs = count(target_directory)

if total_pairs > 0 :
    print(f"{target_directory} 폴더 내의 QA쌍 개수 : {total_pairs}")

./data/HarryPotterQA 폴더 내의 QA쌍 개수 : 28888
